In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator

from matplotlib.ticker import FormatStrFormatter

import scipy.stats as sstat
import scipy.signal as ssig
import h5py
from mpl_toolkits.mplot3d import Axes3D
import os
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA as sklearnPCA
import re

# import ephys_unit_analysis as ena
import mz_ephys_unit_analysis as mz_ena

#import resampy
import fnmatch
import seaborn as sns
%matplotlib inline
%load_ext autoreload
%autoreload 2
mpl.rcParams['pdf.fonttype'] = 42 
mpl.rcParams['font.sans-serif']=['Arial', 'Helvetica','Bitstream Vera Sans', 'DejaVu Sans', 'Lucida Grande', 
                                 'Verdana', 'Geneva', 'Lucid', 'Avant Garde', 'sans-serif']  

sns.despine()

In [2]:
# for publication quality plots
def set_pub_plots(pal=sns.blend_palette(["gray","crimson", 'cyan', 'magenta', 'purple'  ],5)):
    sns.set_style("white")
    sns.set_palette(pal)
    sns.set_context("poster", font_scale=1.5, rc={"lines.linewidth": 2.5, "axes.linewidth":2.5, 'figure.facecolor': 'white'}) 
    sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
    plt.rcParams['axes.linewidth'] = 2.5

rc_pub={'font.size': 25, 'axes.labelsize': 25, 'legend.fontsize': 25.0, 
        'axes.titlesize': 25, 'xtick.labelsize': 25, 'ytick.labelsize': 25, 
        'axes.linewidth':2.5, 'lines.linewidth': 2.5,
        'xtick.color': 'black', 'ytick.color': 'black', 'axes.edgecolor': 'black',
        'axes.labelcolor':'black','text.color':'black'}
# to restore the defaults, call plt.rcdefaults() 

set_pub_plots()

In [ ]:
pal=sns.blend_palette(['black','royalblue'], 2)
sns.palplot(pal)
sns.set_palette(pal)

---

# First, load in the previously saved dataFrame

In [ ]:
final_df = pd.read_pickle(r"D:\mz_Data\saved_dfs\HPC_nmda\psth_df.pkl")

print(final_df.et.nunique())
print(final_df.group.unique())
print(final_df.region.unique())
print(final_df.situ.unique())
print(final_df.situ.unique())

final_df.head()

---

# Subplot unit heatmaps
Requires a __`user input`__ to choose the stimulus situation

In [ ]:
brain_region = 'v1'
stim_cond = input('pre - post - novel: ')   #(pre, post, novel)
is_sorted = 'yes'    #(yes, no)

In [38]:
V1_all_groups_df = final_df[final_df['region'] == brain_region]
V1_all_groups_situ = V1_all_groups_df[V1_all_groups_df['situ'] == stim_cond]

V1_A = V1_all_groups_situ[V1_all_groups_situ['group'] == 'sham']
V1_B = V1_all_groups_situ[V1_all_groups_situ['group'] == 'nmda']

In [39]:
A_hm1 = V1_A.pivot('cuid', 'times', 'zscore')
B_hm1 = V1_B.pivot('cuid', 'times', 'zscore')

if is_sorted == 'yes':
    A_hm1_np = A_hm1.values[ np.argsort(np.mean(A_hm1.values[:,50:70], axis = 1) )] #if I want sorted activity
    B_hm1_np = B_hm1.values[ np.argsort(np.mean(B_hm1.values[:,50:70], axis = 1) )] #if I want sorted activity
elif is_sorted == 'no':
    A_hm1_np = A_hm1.values
    B_hm1_np = B_hm1.values

# Finally, plot it

In [40]:
sp_dims = (15, 5) #(width, height)
v_max = 4
v_min = -1
xvalues = np.arange(start=0, stop=3, step=0.5)
y_min = -2
y_max = 6

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2, sharex='col', figsize=sp_dims)
f.tight_layout()
plt.suptitle(stim_cond)

# Heatmap for A
sns.heatmap(A_hm1_np, 
            cmap = 'afmhot', xticklabels=50, yticklabels=False, 
            vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax1)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel(str(len(V1_A.cuid.unique())))
ax1.set_xticklabels(xvalues, rotation=0)
ax1.set_xlim(0,2.9*100)
ax1.set_title('sham')
cbar = ax1.collections[0].colorbar
cbar.remove()

#Heatmap for B
sns.heatmap(B_hm1_np, 
            cmap = 'afmhot', xticklabels=50, yticklabels=False, 
            vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax2)
ax2.set_xlabel('Time (s)')
ax2.set_ylabel(str(len(V1_B.cuid.unique())))
ax2.set_xticklabels(xvalues, rotation=0)
ax2.set_xlim(0,2.9*100)
ax2.set_title('nmda')
cbar = ax2.collections[0].colorbar
cbar.ax.tick_params(labelsize=25)
cbar.set_label('Z-score')

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\units\all_novel_heat.pdf", transparent=True)

plt.show()

In [ ]:
#Combo lineplot
g = sns.relplot(x = 'times', y = 'zscore', 
             estimator = 'mean', 
             data = V1_all_groups_df, 
             hue = 'group',
             col='situ',
             kind='line',
             hue_order = ['sham','nmda'],
             legend=False,
             ci=None) #ci is the error

(g.map(plt.axhline, y=0, color=".7", dashes=(2, 1), zorder=0)
  .map(plt.axvspan, xmin=0.5,xmax=0.7, color='Gray', alpha=0.25)
  .set_axis_labels("Time (s)", "Z-score")
  .set_titles("{col_name}")
  .set(ylim=(-1.5, 2.5), yticks=([-1,0,1,2]), xlim=(0.0,2.9), xticks=([0,0.5,1,1.5,2,2.5]))
  .tight_layout(w_pad=0)
)

# plt.legend(['sham','nmda'], loc="upper right")

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\units\all_line.pdf", transparent=True)

plt.show()

---

# Plotting only the visually responsive units

In [42]:
def vis_resp_units(final_df, region='v1', time_window=[0.5, 0.9], z_score = 1.0):
    
    test_df = final_df[final_df['region'] == region]
    
    vis_resp = test_df[(test_df['times'] >= time_window[0]) 
                       & (test_df['times'] <= time_window[1]) 
                       & (test_df['zscore'] > z_score)]
    
    test2 = test_df[test_df.cuid.isin(vis_resp.cuid.unique())]
    
    return test2
    

In [ ]:
brain_region = 'v1'
stim_cond = input('pre - post - novel: ')   #(pre, post, novel)
is_sorted = 'yes'    #(yes, no)

V1_all_groups_df = final_df[final_df['region'] == brain_region]
V1_all_groups_situ = V1_all_groups_df[V1_all_groups_df['situ'] == stim_cond]
V1_A = V1_all_groups_situ[V1_all_groups_situ['group'] == 'sham']
V1_B = V1_all_groups_situ[V1_all_groups_situ['group'] == 'nmda']

A_hm1 = V1_A.pivot('cuid', 'times', 'zscore')
B_hm1 = V1_B.pivot('cuid', 'times', 'zscore')

if is_sorted == 'yes':
    A_hm1_np = A_hm1.values[ np.argsort(np.mean(A_hm1.values[:,50:70], axis = 1) )] #if I want sorted activity
    B_hm1_np = B_hm1.values[ np.argsort(np.mean(B_hm1.values[:,50:70], axis = 1) )] #if I want sorted activity
elif is_sorted == 'no':
    A_hm1_np = A_hm1.values
    B_hm1_np = B_hm1.values

In [56]:
vis_resp_df = vis_resp_units(V1_all_groups_situ, region=brain_region)

plot_A = vis_resp_df[vis_resp_df['group'] == 'sham']
plot_B = vis_resp_df[vis_resp_df['group'] == 'nmda']

test_A = plot_A.pivot('cuid', 'times', 'zscore')
test_A_sort = test_A.values[ np.argsort(np.mean(test_A.values[:,50:70], axis = 1) )] #if I want sorted activity

test_B = plot_B.pivot('cuid', 'times', 'zscore')
test_B_sort = test_B.values[ np.argsort(np.mean(test_B.values[:,50:70], axis = 1) )] #if I want sorted activity

In [57]:
sp_dims = (15, 6) #(width, height)
v_max = 4
v_min = -1
xvalues = np.arange(start=0, stop=3, step=0.5)
y_min = -2
y_max = 6

plt_title = str(stim_cond)

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2, sharex='col', figsize=sp_dims)
f.tight_layout()
plt.suptitle(plt_title)

# Heatmap for A
sns.heatmap(test_A_sort, cmap = 'afmhot', xticklabels=50, yticklabels=False, 
                 vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax1)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel(str(len(plot_A.cuid.unique())))
ax1.set_xticklabels(xvalues, rotation=0)
ax1.set_xlim(0,2.9*100)
ax1.set_title('sham')
cbar = ax1.collections[0].colorbar
cbar.remove()

#Heatmap for B
sns.heatmap(test_B_sort, cmap = 'afmhot', xticklabels=50, yticklabels=False, 
                 vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax2)
ax2.set_xlabel('Time (s)')
ax2.set_ylabel(str(len(plot_B.cuid.unique())))
ax2.set_xticklabels(xvalues, rotation=0)
ax2.set_xlim(0,2.9*100)
ax2.set_title('nmda')
cbar = ax2.collections[0].colorbar
cbar.ax.tick_params(labelsize=25)
cbar.set_label('Z-score')

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\units\vr_novel_heat.pdf", transparent=True)

plt.show()

In [ ]:
# Lineplot
sns.lineplot(x = 'times', y = 'zscore', 
             estimator = 'mean', 
             data = vis_resp_df, 
             hue = 'group',
             hue_order = ['sham', 'nmda'],
             legend=False,
             ci=None) #ci is the error

plt.legend(['sham','nmda'], loc="upper right")

plt.axhline(y=0, xmin=0, xmax=3, c='black', linewidth=0.5, zorder=0)
plt.axvspan(0.5,0.7, color='Gray', alpha=0.25)
plt.tick_params(axis='x', rotation=0)
plt.xlim(0,2.9)
plt.xticks([0,0.5,1,1.5,2,2.5])
plt.ylim(-1,3)
plt.yticks([-1,0,1,2,3])

plt.title(plt_title)
plt.xlabel('Time(s)')
plt.ylabel('Z-score')

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\units\vr_novel_line.pdf", transparent=True)

plt.show()

---

---